<font size=6>**面向对象高级编程**<font/>

数据封装、继承和多态只是面向对象程序设计中最基础的3个概念。在Python中，面向对象还有很多高级特性，允许我们写出非常强大的功能。

我们会讨论多重继承、定制类、元类等概念。

# 使用\_\_slots\_\_
正常情况下，当我们定义了一个class，创建了一个class的实例后，我们可以给该实例绑定任何属性和方法，这就是动态语言的灵活性。先定义class：

In [1]:
class Student(object):
    pass

然后尝试给实例绑定属性

In [2]:
s=Student()
s.name='Micheal'
print(s.name)

Micheal


还可以尝试给实例绑定一个方法。

In [3]:
def set_age(self,age):
    self.age=age

In [5]:
from types import MethodType
s.set_age=MethodType(set_age,s)#给实例绑定方法
s.set_age(10)
print(s.age)

10


但是，给一个实例绑定的方法，对另一个实例是不起作用的

In [6]:
s2=Student()
s2.set_age(25)

AttributeError: 'Student' object has no attribute 'set_age'

为了给所有实例都绑定方法，可以给class绑定方法：

In [7]:
def set_score(self,score):
    self.score=score

In [19]:
Student.set_score=set_score #动态绑定类方法可以直接赋值

给类绑定方法后，所有实例都可调用。

In [9]:
s.set_score(70)
s2.set_score(87)
s2.score

87

In [10]:
def set_gender(self,gender):
    self.gender=gender

In [11]:
s.set_gender=set_gender
s.set_gender('male')
s.gender

TypeError: set_gender() missing 1 required positional argument: 'gender'

In [12]:
s.set_gender(s,'male')
s.gender

'male'

使用赋值语句和MethodType均可以绑定方法。
但是使用MethodType绑定方法，调用方法时会将被绑定对象作为参数自动传入方法。
使用赋值语句绑定方法，不会自动传参。但是值得注意的是，如果是类绑定方法，那么他自身使用该方法时不会自动传参，但是他的对象也拥有该方法，而且使用时会自动传参。

___

但是，如果我们想要限制实例的属性怎么办？比如，只允许对Student实例添加name和age属性。

为了达到限制的目的，Python允许在定义class的时候，定义一个特殊的\_\_slots\_\_变量，来限制该class实例能添加的属性：

In [13]:
class Student(object):
    __slots__=('name','age') # 用tuple定义允许绑定的属性名称

In [14]:
s=Student()
s.name='Michael'
s.age=25
s.score=78

AttributeError: 'Student' object has no attribute 'score'

由于'score'没有被放到\_\_slots\_\_中，所以不能绑定score属性，试图绑定score将得到AttributeError的错误。

使用\_\_slots\_\_要注意，\_\_slots\_\_定义的属性**仅对当前类实例起作用**，对继承的子类是不起作用的：

In [15]:
class GraduateStudent(Student):
    pass

In [16]:
g=GraduateStudent()
g.score=100
g.score

100

除非在子类中也定义\_\_slots\_\_，这样，子类实例允许定义的属性就是自身的\_\_slots\_\_加上父类的\_\_slots\_\_

In [17]:
class SmallStudent(Student):
    __slots__=('score','phonenumber')

In [18]:
small_student=SmallStudent()
small_student.name='Michael'
small_student.age=11
small_student.score=89
small_student.phonenumber='172681'
small_student.gender='male'

AttributeError: 'SmallStudent' object has no attribute 'gender'

## 小练习
* s.set_age=MethodType(set_age,s)和setattr(s,'set_age',set_age)有什么区别？<br/>
setattr是先检查Student类有无 ‘set_age’方法，没有的话再加上去。  

# 使用@property
在绑定属性时，如果我们直接把属性暴露出去，虽然写起来很简单，但是，没办法检查参数，导致可以把成绩随便改：

In [23]:
class Student(object):
    pass

s=Student()
s.score=999

这显然不合逻辑。为了限制score的范围，可以通过一个set_score()方法来设置成绩，再通过一个get_score()来获取成绩，这样，在set_score()方法里，就可以检查参数：

In [24]:
class Student(object):
    def get_score(self):
        return self.score

    def set_score(self,value):
        if not isinstance(value,int):
            raise ValueError("score must be an integer!")
        if value<0 or value>100:
            raise ValueError("score must between 0-100!")
        self.score=value

In [26]:
s=Student()
s.set_score(60)
s.get_score()

60

In [27]:
s.set_score(999)

ValueError: score must between 0-100!

In [28]:
s.score=999

虽然限制了输入，但是可以通过直接赋值进行更改，有没有既能检查参数，又可以用类似属性这样简单的方式来访问类的变量呢？
还记得装饰器（decorator）可以给函数动态加上功能吗？对于类的方法，装饰器一样起作用。Python内置的@property装饰器就是负责把一个方法变成属性调用的：

In [31]:
class Student(object):
    
    @property
    def score(self):
        return self._score
    
    @score.setter
    def score(self,value):
        if not isinstance(value,int):
            raise ValueError("score must be an integer!")
        if value<0 or value>100:
            raise ValueError("score must between 0-100!")
        self._score=value

@property的实现比较复杂，我们先考察如何使用。把一个getter方法变成属性，只需要加上@property就可以了，此时，@property本身又创建了另一个装饰器@score.setter，负责把一个setter方法变成属性赋值，于是，我们就拥有一个可控的属性操作：

In [32]:
s=Student()
s.score=60
s.score

60

In [33]:
s.score=999

ValueError: score must between 0-100!

注意到这个神奇的@property，我们在对实例属性操作的时候，就知道该属性很可能不是直接暴露的，而是通过getter和setter方法来实现的。

还可以定义只读属性，只定义getter方法，不定义setter方法就是一个只读属性：

In [34]:
class Student(object):
    
    @property
    def birth(self):
        return self._birth
    
    @birth.setter
    def birth(self,value):
        self._birth=value
    
    @property
    def age(self):
        return 2015-self.birth
    

上面的birth是可读写属性，而age就是一个只读属性，因为age可以根据birth和当前时间计算出来。

In [35]:
s=Student()
s.birth=1995

In [36]:
s.age

20

In [37]:
s.birth

1995

* 练习：
请利用@property给一个Screen对象加上width和height属性，以及一个只读属性resolution：

In [40]:
class Screen(object):
    
    @property
    def width(self):
        return self._width
    @width.setter
    def width(self,value):
        if not isinstance(value,int):
            raise ValueError("value must be integer")
        if 0>value or value>1200:
            raise ValueError('value must between 0-1200')
        self._width=value
    
    @property
    def height(self):
        return self._height
    @height.setter
    def height(self,value):
        if not isinstance(value,int):
            raise ValueError("value must be integer")
        if 0>value or value>1000:
            raise ValueError('value must between 0-1000')
        self._height=value
    
    @property
    def resolution(self):
        return self._width*self._height

In [41]:
# 测试:
s = Screen()
s.width = 1024
s.height = 768
print('resolution =', s.resolution)
if s.resolution == 786432:
    print('测试通过!')
else:
    print('测试失败!')

resolution = 786432
测试通过!


# 多重继承

回忆一下Animal类层次的设计，假设我们要实现以下4种动物：

* Dog - 狗狗；
* Bat - 蝙蝠；
* Parrot - 鹦鹉；
* Ostrich - 鸵鸟。

采用多重继承。首先，主要的类层次仍按照哺乳类和鸟类设计：

In [44]:
class Animal(object):
    pass

#大类
class Mammal(Animal):
    pass

class Bird(Animal):
    pass

#各种动物
class Dog(Mammal):
    pass

class Bat(Mammal):
    pass

class Parrot(Bird):
    pass

class Ostrich(Bird):
    pass


现在，我们要给动物再加上Runnable和Flyable的功能，只需要先定义好Runnable和Flyable的类：

In [45]:
class Runnable(object):
    def run(self):
        print("Running...")

class Flyable(object):
    def fly(self):
        print('Flying...')

对于需要Runnable功能的动物，就多继承一个Runnable，例如Dog：

In [46]:
class Dog(Mammal,Runnable):
    pass

对于需要Flyable功能的动物，就多继承一个Flyable，例如Bat：

In [48]:
class Bat(Mammal,Flyable):
    pass

通过多重继承，一个子类就可以同时获得多个父类的所有功能。

在设计类的继承关系时，通常，主线都是单一继承下来的，例如，Ostrich继承自Bird。但是，如果需要“混入”额外的功能，通过多重继承就可以实现，比如，让Ostrich除了继承自Bird外，再同时继承Runnable。这种设计通常称之为**MixIn**。

为了更好地看出继承关系，我们把Runnable和Flyable改为RunnableMixIn和FlyableMixIn。类似的，你还可以定义出肉食动物CarnivorousMixIn和植食动物HerbivoresMixIn，让某个动物同时拥有好几个MixIn：

In [49]:
class RunnableMixln(object):
    def run(self):
        print("Running...")

class FlyableMixln(object):
    def fly(self):
        print('Flying...')

class CarnivorousMixln(object):
    def eat_meat(self):
        print("Eatting meat...")

class HerbivoresMixln(object):
    def eat_plant(self):
        print('Eatting plant...')

In [50]:
class Dog(Mammal,RunnableMixln,CarnivorousMixln,HerbivoresMixln):
    pass

MixIn的目的就是给一个类增加多个功能，这样，在设计类的时候，我们优先考虑通过多重继承来组合多个MixIn的功能，而不是设计多层次的复杂的继承关系。

In [52]:
d=Dog()
d.run()
d.eat_meat()
d.eat_plant()
d.fly()

Running...
Eatting meat...
Eatting plant...


AttributeError: 'Dog' object has no attribute 'fly'

Python自带的很多库也使用了MixIn。举个例子，Python自带了TCPServer和UDPServer这两类网络服务，而要同时服务多个用户就必须使用多进程或多线程模型，这两种模型由ForkingMixIn和ThreadingMixIn提供。通过组合，我们就可以创造出合适的服务来。

比如，编写一个多进程模式的TCP服务，定义如下：

In [53]:
class MyTCPServer(TCPSerer,ForkingMixln):
    pass

NameError: name 'TCPSerer' is not defined

编写一个多线程UDP服务：

In [54]:
class MyUDPServer(UDPServer,ThreadingMixln):
    pass

NameError: name 'UDPServer' is not defined

如果你打算搞一个更先进的协程模型，可以编写一个CoroutineMixIn：

In [55]:
class MyTCPServer(TCPServer, CoroutineMixIn):
    pass

NameError: name 'TCPServer' is not defined

这样一来，我们不需要复杂而庞大的继承链，只要选择组合不同的类的功能，就可以快速构造出所需的子类。

# 定制类
看到类似\_\_slots\_\_这种形如\_\_xxx\_\_的变量或者函数名就要注意，这些在Python中是有特殊用途的。

\_\_slots\_\_我们已经知道怎么用了，\_\_len\_\_()方法我们也知道是为了能让class作用于len()函数。

除此之外，Python的class中还有许多这样有特殊用途的函数，可以帮助我们定制类

## \_\_str\_\_
用于重定义打印print

In [56]:
class Student(object):
    def __init__(self,name):
        self.name=name
s=Student('Michael')
print(s)

怎么才能打印得好看呢？只需要定义好\_\_str\_\_()方法，作用于print()函数返回一个好看的字符串就可以了：

In [57]:
class Student(object):
    def __init__(self,name):
        self.name=name
    def __str__(self):
        return 'Student object (name:%s)' % self.name
s=Student('Michael')
print(s)

Student object (name:Michael)


但是细心的朋友会发现直接敲变量不用print，打印出来的实例还是不好看：

In [58]:
s

这是因为直接显示变量调用的不是\_\_str\_\_()，而是\_\_repr\_\_()，两者的区别是\_\_str\_\_()返回用户看到的字符串，而\_\_repr\_\_()返回程序开发者看到的字符串，也就是说，\_\_repr\_\_()是为调试服务的。

解决办法是再定义一个\_\_repr\_\_()。但是通常\_\_str\_\_()和\_\_repr\_\_()代码都是一样的，所以，有个偷懒的写法：

In [59]:
class Student(object):
    def __init__(self,name):
        self.name=name
    def __str__(self):
        return 'Student object (name:%s)' % self.name
    __repr__=__str__
    
s=Student('Michael')
print(s)
s

Student object (name:Michael)


Student object (name:Michael)

In [62]:
print('hello\n')
repr('hello\n')

hello



"'hello\\n'"

## \_\_iter\_\_

如果一个类想被用于for ... in循环，类似list或tuple那样，就必须实现一个\_\_iter\_\_()方法，该方法返回一个迭代对象，然后，Python的for循环就会不断调用该迭代对象的\_\_next\_\_()方法拿到循环的下一个值，直到遇到StopIteration错误时退出循环。

我们以斐波那契数列为例，写一个Fib类，可以作用于for循环：

In [65]:
class Fib(object):
    def __init__(self):
        self.a,self.b=0,1
        
    def __iter__(self):
        return self  # 实例本身就是迭代对象，故返回自己
    
    def __next__(self):
        self.a,self.b=self.b,self.a+self.b
        if self.a>100000:
            raise StopIteration()
        return self.a

In [66]:
for n in Fib():
    print(n)

1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946
17711
28657
46368
75025


## \_\_getitem\_\_
Fib实例虽然能作用于for循环，看起来和list有点像，但是，把它当成list来使用还是不行，比如，取第5个元素：

In [67]:
Fib()[5]

TypeError: 'Fib' object does not support indexing

要表现得像list那样按照下标取出元素，需要实现\_\_getitem\_\_()方法：

In [69]:
class Fib(object):
    def __init__(self):
        self.a,self.b=0,1
        
    def __iter__(self):
        return self  # 实例本身就是迭代对象，故返回自己
    
    def __next__(self):
        self.a,self.b=self.b,self.a+self.b
        if self.a>100000:
            raise StopIteration()
        return self.a
    def __getitem__(self,n):
        a,b=0,1
        for x in range(n):
            a,b=b,a+b
        return a

In [70]:
Fib()[5]

5

In [71]:
Fib()[100]

354224848179261915075

但是list有个神奇的切片方法：

In [72]:
list(range(10))[3:7]

[3, 4, 5, 6]

对于Fib却报错。原因是__getitem__()传入的参数可能是一个int，也可能是一个切片对象slice，所以要做判断：

In [73]:
Fib()[3:6]

TypeError: 'slice' object cannot be interpreted as an integer

In [74]:
class Fib(object):
    def __init__(self):
        self.a,self.b=0,1
        
    def __iter__(self):
        return self  # 实例本身就是迭代对象，故返回自己
    
    def __next__(self):
        self.a,self.b=self.b,self.a+self.b
        if self.a>100000:
            raise StopIteration()
        return self.a
    
    def __getitem__(self,n):
        if isinstance(n,int):# n是索引
            a,b=1,1
            for x in range(n):
                a,b=b,a+b
            return a
        if isinstance(n,slice): # n是切片
            start=n.start
            stop=n.stop
            if start is None:
                start=0
            a,b=1,1
            L=[]
            for x in range(stop):
                if x>=start:
                    L.append(a)
                a,b=b,a+b
            return L

In [75]:
Fib()[:5]

[1, 1, 2, 3, 5]

In [76]:
Fib()[2:5]

[2, 3, 5]

但是没有对step参数作处理：

In [79]:
Fib()[:5:2]

[1, 1, 2, 3, 5]

也没有对负数作处理，所以，要正确实现一个\_\_getitem\_\_()还是有很多工作要做的。

此外，如果把对象看成dict，\_\_getitem\_\_()的参数也可能是一个可以作key的object，例如str。

与之对应的是\_\_setitem\_\_()方法，把对象视作list或dict来对集合赋值。最后，还有一个\_\_delitem\_\_()方法，用于删除某个元素。

总之，通过上面的方法，我们自己定义的类表现得和Python自带的list、tuple、dict没什么区别，这完全归功于动态语言的“鸭子类型”，不需要强制继承某个接口。

## \_\_getattr\_\_
正常情况下，当我们调用类的方法或属性时，如果不存在，就会报错。比如定义Student类：

In [80]:
class Student(object):
    def __init__(self):
        self.name='Michael'

In [81]:
s=Student()
s.name

'Michael'

In [82]:
s.score

AttributeError: 'Student' object has no attribute 'score'

要避免这个错误，除了可以加上一个score属性外，Python还有另一个机制，那就是写一个\_\_getattr\_\_()方法，动态返回一个属性。修改如下：

In [83]:
class Student(object):
    def __init__(self):
        self.name='Michael'
    
    def __getattr__(self,attr):
        if attr=='score':
            return 90

In [84]:
s=Student()
s.name

'Michael'

In [85]:
s.score

90

返回函数也是完全可以的：

In [90]:
class Student(object):
    def __init__(self):
        self.name='Michael'
    
    def __getattr__(self,attr):
        if attr=='age':
            return 25
        

In [91]:
s=Student()
s.age()

TypeError: 'int' object is not callable

In [92]:
class Student(object):
    def __init__(self):
        self.name='Michael'
    
    def __getattr__(self,attr):
        if attr=='age':
            return lambda:25
        
s=Student()
s.age()

25

注意，只有在没有找到属性的情况下，才调用\_\_getattr\_\_，已有的属性，比如name，不会在\_\_getattr\_\_中查找。

还可以写抛出AttributeError错误:

In [94]:
class Student(object):
    def __init__(self):
        self.name='Michael'
    
    def __getattr__(self,attr):
        if attr=='age':
            return lambda:25
        raise AttributeError('Student object has no attribute %s' % attr)
        
s=Student()
print(s.age())
s.gender()

25


AttributeError: Student object has no attribute gender

## \_\_call\_\_
一个对象实例可以有自己的属性和方法，当我们调用实例方法时，我们用instance.method()来调用。能不能直接在实例本身上调用呢？在Python中，答案是肯定的。

任何类，只需要定义一个\_\_call\_\_()方法，就可以直接对实例进行调用。请看示例：

In [95]:
class Student(object):
    def __init__(self,name):
        self.name=name
    
    def __call__(self):
        print('Myname is %s ' % self.name)

In [96]:
s=Student('Machael')

In [97]:
s()

Myname is Machael 


\_\_call\_\_()还可以定义参数。对实例进行直接调用就好比对一个函数进行调用一样，所以你完全可以把对象看成函数，把函数看成对象，因为这两者之间本来就没啥根本的区别。

如果你把对象看成函数，那么函数本身其实也可以在运行期动态创建出来，因为类的实例都是运行期创建出来的，这么一来，我们就模糊了对象和函数的界限。

那么，怎么判断一个变量是对象还是函数呢？其实，更多的时候，我们需要判断一个对象是否能被调用，能被调用的对象就是一个Callable对象，比如函数和我们上面定义的带有\_\_call\_\_()的类实例：

In [99]:
callable(Student('Name'))

True

In [100]:
callable([])

False

In [101]:
callable("asd")

False

In [102]:
callable(None)

False

In [103]:
callable(abs)

True

In [104]:
callable(s)

True

通过callable()函数，我们就可以判断一个对象是否是“可调用”对象。